### The Culture of International Relations - Corpus statistics

#### About this notebook
tbd

In [6]:
%%html
<style>
.jupyter-widgets, .widget-dropdown > select, .widget-label, .widget-toggle-button > button {
    font-size: 8pt;
}
</style>

### <span style='color:blue'>**Mandatory Prepare Step**</span>: Setup Notebook
The following code cell must to be executed once for each user session. The step loads utility Python code stored in separate files, and imports dependencies to external libraries.


In [15]:
# Setup

#%run ../common/file_utility
#%run ../common/network_utility
import sys
sys.path.insert(1, '../common')

import widgets_utility
import configuration_elements
import utility
import treaty_state

logger = getLogger('corpus_text_analysis')

%load_ext autoreload
%autoreload 1
%aimport widgets_utility, treaty_state

treaty_state = load_treaty_state(data_folder='../data')

# Multiple notebooks can be attached to same kernal (Change kernal command)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


###  <span style='color:blue'>**Mandatory Step**</span>: Convert Treaty Text Corpora to MM Format

This code cell is a mandatory step for subsequent text corpus statistics. 

This step processes the treaty text for from given compressed archive (ZIP-file), each language , and stores in an efficient Market-Matrix (MM) corpus format. The corpora is only stored if it is not previously stored, or the "Force Update" is specified. Note that an update MUST be forced whenever the treaty archive is updated - otherwise the text in the new archive is ignored.

Corpus 

In [1]:
# Code
%run ./treaty_corpus
def prepare_treaty_text_corpora(source_pattern):
    
    def store_mm_corpora(source_path, force, languages):
        try:
            progress_widget.value = 1
            tokenizer = nltk.tokenize.word_tokenize
            source_folder = os.path.split(source_path)[0]
            for language in languages.split(','):
                progress_widget.value += 1
                loader = TreatyCorpusSaveLoad(source_folder, language)
                if not loader.exists() or force:
                    progress_widget.description = 'Processing: {}'.format(language)
                    stream = CompressedFileReader(source_path, filename_pattern='*_{}*.txt'.format(language))
                    treaty_corpus = TreatyCorpus(stream, tokenizer=tokenizer)        
                    loader.store_as_mm_corpus(treaty_corpus)
            progress_widget.description = ''
            progress_widget.value = 0
        except Exception as ex:
            logger.error(ex)
        
    current_archives = (ls_sorted(source_pattern) or [])

    source_path_widget=widgets.Dropdown(
        options=current_archives,
        value=current_archives[-1] if len(current_archives) else None,
        description='Corpus:' #, **drop_style
    )
    force_corpus_update_widget=widgets.ToggleButton(
        description='Force Update',
        tooltip='Force refresh of saved corpus cache (a performance feature).\
        Use when ZIP-archive has been updated.',
        value=False #, **toggle_style
    )
    progress_widget = widgets.IntProgress(min=0, max=5, step=1, value=0)

    wi = widgets.interactive(
        store_mm_corpora,
        source_path=source_path_widget,
        force=force_corpus_update_widget,
        languages='en,it,fr,de'
    )

    display(
        widgets.VBox([widgets.HBox([source_path_widget, force_corpus_update_widget, progress_widget]), wi.children[-1]])
    )

    wi.update()
    
prepare_treaty_text_corpora('../data/*.zip')


NameError: name 'widgets' is not defined

In [5]:
import re
from IPython.display import display, HTML

%run ./treaty_corpus
%run ../common/treaty_state

treaty_state = load_treaty_state(data_folder='../data')

def specify_corpus_reader(archive_filename, corpus_option, language, caption, treaty_state=None):
    
    def get_document_names(archive_filename):
        document_names = CompressedFileReader(zip_archive=archive_filename).archive_filenames
        df = pd.DataFrame([[x] + re.split('_|\.',x)[:2] for x in document_names],
                          columns=['filename', 'treaty_id', 'language'])
        df = df.set_index('treaty_id')
        df['is_corr'] = df.filename.apply(lambda z: "_corr" in z)
        return df
    
    if corpus_option is None:
        caption.value = 'Please select corpus'
        return

    df_documents = get_document_names(archive_filename)
    df_treaties = treaty_state.get_treaty_subset(corpus_option, language).join(df_documents)

    display(HTML(df_treaties.to_html()))

    return df_treaties
    
def specify_corpus_reader_interactor(source_pattern, treaty_state):
    
    current_archives = (ls_sorted(source_pattern) or [])
    
    progress_widget=widgets.IntProgress(min=0, max=5, step=1, value=0, layout=widgets.Layout(width='99%'))

    archive_filename_widget=widgets.Dropdown(
        options=current_archives,
        value=current_archives[-1] if len(current_archives) else None,
        description='Text archive:' #, **drop_style
    )
      
    corpus_options = {
        'UNTS 1946-1972': dict(source=['UNTS', 'UNXX'], from_year=1946, to_year=1972,
                               parties=None, lang=['en', 'fr']),
        'Party of Five': dict(source=None, from_year=1915, to_year=1972,
                              parties=parties_of_interest, lang=['en', 'fr', 'de', 'it'])
    }
    
    corpus_widget=widgets.Dropdown(
        options=corpus_options,
        value=corpus_options[list(corpus_options.keys())[0]],
        description='Corpus:'
    )

    language_widget=widgets.Dropdown(
        options=['en', 'fr', 'it', 'de'],
        value='en',
        description='Language:'
    )
    
    caption = widgets.Label(value='Please specify corpus')
    
    w = widgets.interactive(
        specify_corpus_reader,
        archive_filename=archive_filename_widget,
        corpus_option=corpus_widget,
        language=language_widget,
        caption=caption,
        treaty_state=widgets.fixed(treaty_state)
    )

    display(
        widgets.VBox([
            widgets.HBox([caption]),
            widgets.HBox([archive_filename_widget, corpus_widget, language_widget]),
            widgets.HBox([progress_widget]),
            w.children[-1]])
    )
    
    #def corpus_widget_change(change):
    #    current_value = language_widget.value
    #    languages = None if corpus_widget.value is None else corpus_widget.value['lang']
    #    language_widget.options = languages or []
    #    caption.value = str(lang)
                 
    #corpus_widget.observe(corpus_widget_change, names='value')

    return w

treaties_select_widget = specify_corpus_reader_interactor('../data/*.zip', treaty_state)
treaties_select_widget.update()

#print(treaties_select_widget)
#def data_changed(change):
#    raise Exception("HEJ")
#    
#treaties_select_widget.observe(data_changed, names='value')


### Task: Basic Corpus Statistics
See https://www.nltk.org/book/ch01.html

* Size of treaties over time
* Unique word, unique words per word class
* Lexical diversity
* Frequency distribution
* Average word length, sentence length


```python
 	
>>> len(texts) / count(docs)
0.06230453042623537
>>>

>>> len(set(text3)) / len(text3)
0.06230453042623537
>>>

>>> > def lexical_diversity(text): [1]
...     return len(set(text)) / len(text) [2]
...
>>> def percentage(count, total): [3]
...     return 100 * count / total

# Most common words
fdist1 = FreqDist(text1)
fdist1.most_common(50)

# Word length frequencies
>>> fdist = FreqDist(len(w) for w in text1)  [2]
>>> print(fdist)

```


In [17]:
# Code 

corpus = None
def display_token_toplist_interact(source_folder):
    global corpus
    progress_widget = None
    
    def display_token_toplist(source_folder, language, statistics='', remove_stopwords=False):
        global corpus

        try:

            progress_widget.value = 1

            corpus = TreatyCorpusSaveLoad(source_folder=source_folder, lang=language[0]).load_mm_corpus()

            progress_widget.value = 2
            service = MmCorpusStatisticsService(corpus, dictionary=corpus.dictionary, language=language)

            print("Corpus consists of {} documents, {} words in total and a vocabulary size of {} tokens."\
                      .format(len(corpus), corpus.dictionary.num_pos, len(corpus.dictionary)))

            progress_widget.value = 3
            if statistics == 'word_freqs':
                display(service.compute_word_frequencies(remove_stopwords))
            elif statistics == 'documents':
                display(service.compute_document_stats())
            elif statistics == 'word_count':
                display(service.compute_word_stats())
            else:
                print('Unknown: ' + statistics)

        except Exception as ex:
            logger.error(ex)

        progress_widget.value = 5
        progress_widget.value = 0
        return corpus
    
    language_widget=widgets.Dropdown(
        options={
            'English': ('en', 'english'),
            'French': ('fr', 'french'),
            'German': ('de', 'german'),
            'Italian': ('it', 'italian')
        },
        value=('en', 'english'),
        description='Language:', **dict(layout=widgets.Layout(width='260px'))
    )
    
    statistics_widget=widgets.Dropdown(
        options={
            'Word freqs': 'word_freqs',
            'Documents': 'documents',
            'Word count': 'word_count'
        },
        value='word_count',
        description='Statistics:', **dict(layout=widgets.Layout(width='260px'))
    )
    
    remove_stopwords_widget=widgets.ToggleButton(
        description='Remove stopwords', value=True,
        tooltip='Do not include stopwords in token toplist'
    )
    
    progress_widget=wf.create_int_progress_widget(min=0, max=5, step=1, value=0) #, layout=widgets.Layout(width='100%')),

    wi = widgets.interactive(
        display_token_toplist,
        source_folder=source_folder,
        language=language_widget,
        statistics=statistics_widget,
        remove_stopwords=remove_stopwords_widget
    )

    boxes = widgets.HBox(
        [
            language_widget, statistics_widget, remove_stopwords_widget, progress_widget
        ]
    )
    display(widgets.VBox([boxes, wi.children[-1]]))
    wi.update()

display_token_toplist_interact('../data')

In [16]:
filename_predicate = None
filename_predicate = filename_predicate or (lambda x: True)

### <span style='color: red'>WORK IN PROGRESS</span> Task: Treaty Keyword Extraction (using TF-IDF weighing)
- [ML Wiki.org](http://mlwiki.org/index.php/TF-IDF)
- [Wikipedia](https://en.wikipedia.org/wiki/Tf%E2%80%93idf)
- Spärck Jones, K. (1972). "A Statistical Interpretation of Term Specificity and Its Application in Retrieval".
- Manning, C.D.; Raghavan, P.; Schutze, H. (2008). "Scoring, term weighting, and the vector space model". ([PDF](http://nlp.stanford.edu/IR-book/pdf/06vect.pdf))
- https://markroxor.github.io/blog/tfidf-pivoted_norm/
$\frac{tf-idf}{\sqrt(rowSums( tf-idf^2 ) )}$
- https://nlp.stanford.edu/IR-book/html/htmledition/pivoted-normalized-document-length-1.html

Neural Network Methods in Natural Language Processing, Yoav Goldberg:
![image.png](attachment:image.png)

In [18]:
# Code
from scipy.sparse import csr_matrix
%timeit

    
def get_top_tfidf_words(data, n_top=5):
    top_list = data.groupby(['treaty_id'])\
        .apply(lambda x: x.nlargest(n_top, 'score'))\
        .reset_index(level=0, drop=True)
    return top_list

def compute_tfidf_scores(corpus, dictionary, smartirs='ntc'):
    #model = gensim.models.logentropy_model.LogEntropyModel(corpus, normalize=True)
    model = gensim.models.tfidfmodel.TfidfModel(corpus, dictionary=dictionary, normalize=True) #, smartirs=smartirs)
    rows, cols, scores = [], [], []
    for r, document in enumerate(corpus): 
        vector = model[document]
        c, v = zip(*vector)
        rows += (len(c) * [ int(r) ])
        cols += c
        scores += v
        
    return csr_matrix((scores, (rows, cols)))
    
if True: #'tfidf_cache' not in globals():
    tfidf_cache = {
    }
    
def display_tfidf_scores(source_folder, language, period, n_top=5, threshold=0.001):
    
    global state, tfw, tfidf_cache
    
    try:
        treaties = state.treaties

        tfw.progress.value = 0
        tfw.progress.value += 1
        if language[0] not in tfidf_cache.keys():
            corpus = TreatyCorpusSaveLoad(source_folder=source_folder, lang=language[0])\
                .load_mm_corpus(normalize_by_D=True)
            document_names = corpus.document_names
            dictionary = corpus.dictionary
            _ = dictionary[0]

            tfw.progress.value += 1
            A = compute_tfidf_scores(corpus, dictionary)

            tfw.progress.value += 1
            scores = pd.DataFrame(
                [ (i, j, dictionary.id2token[j], A[i, j]) for i, j in zip(*A.nonzero())],
                columns=['document_id', 'token_id', 'token', 'score']
            )
            tfw.progress.value += 1
            scores = scores.merge(document_names, how='inner', left_on='document_id', right_index=True)\
                .drop(['document_id', 'token_id', 'document_name'], axis=1)

            scores = scores[['treaty_id', 'token', 'score']]\
                .sort_values(['treaty_id', 'score'], ascending=[True, False])

            tfidf_cache[language[0]] = scores

        scores = tfidf_cache[language[0]]
        if threshold > 0:
            scores = scores.loc[scores.score >= threshold]

        tfw.progress.value += 1

        #scores = get_top_tfidf_words(scores, n_top=5)
        #scores = scores.groupby(['treaty_id']).sum() 

        scores = scores.groupby(['treaty_id'])\
            .apply(lambda x: x.nlargest(n_top, 'score'))\
            .reset_index(level=0, drop=True)\
            .set_index('treaty_id')

        if period is not None:
            periods = state.treaties[period]
            scores = scores.merge(periods.to_frame(), left_index=True, right_index=True, how='inner')\
                .groupby([period, 'token']).score.agg([np.mean])\
                .reset_index().rename(columns={0:'score'}) #.sort_values('token')

        #['token'].apply(' '.join)

        display(scores)
    except Exception as ex:
        logger.error(ex)
        
    tfw.progress.value = 0

#if 'tfidf_scores' not in globals():
#    tfidf_scores = compute_document_tfidf(corpus, corpus.dictionary, state.treaties)
#    tfidf_scores = tfidf_scores.sort_values(['treaty_id', 'score'], ascending=[True, False])

tfw = BaseWidgetUtility(
    language=widgets.Dropdown(
        options={
            'English': ('en', 'english'),
            'French': ('fr', 'french'),
            'German': ('de', 'german'),
            'Italian': ('it', 'italian')
        },
        value=('en', 'english'),
        description='Language:', **drop_style
    ),
    remove_stopwords=widgets.ToggleButton(
        description='Remove stopwords', value=True,
        tooltip='Do not include stopwords in token toplist', **toggle_style
    ),    
    n_top=widgets.IntSlider(
        value=5, min=1, max=25, step=1,
        description='Top #:',
        continuous_update=False
    ),
    threshold=widgets.FloatSlider(
        value=0.001, min=0.0, max=0.5, step=0.01,
        description='Threshold:',
        tooltip='Word having a TF-IDF score below this value is filtered out',
        continuous_update=False,
        readout_format='.3f',
    ), 
    period=widgets.Dropdown(
        options={
            '': None,
            'Year': 'signed_year',
            'Default division': 'signed_period',
            'Alt. division': 'signed_period_alt'
        },
        value='signed_period',
        description='Period:', **drop_style
    ),
    output=widgets.Dropdown(
        options={
            '': None,
            'Year': 'signed_year',
            'Default division': 'signed_period',
            'Alt. division': 'signed_period_alt'
        },
        value='signed_period',
        description='Output:', **drop_style
    ),
    progress=widgets.IntProgress(min=0, max=5, step=1, value=0) #, layout=widgets.Layout(width='100%')),
)

itfw = widgets.interactive(
    display_tfidf_scores,
    source_folder='./data',
    language=tfw.language,
    n_top=tfw.n_top,
    threshold=tfw.threshold,
    period=tfw.period
)

boxes = widgets.HBox(
    [
        widgets.VBox([tfw.language, tfw.period]),
        widgets.VBox([tfw.n_top, tfw.threshold]),
        widgets.VBox([tfw.progress, tfw.output])
    ]
)

display(widgets.VBox([boxes, itfw.children[-1]]))
itfw.update()


VBox(children=(HBox(children=(VBox(children=(Dropdown(description='Language:', index=3, layout=Layout(width='260px'), options={'German': ('de', 'german'), 'French': ('fr', 'french'), 'Italian': ('it', 'italian'), 'English': ('en', 'english')}, value=('en', 'english')), Dropdown(description='Period:', index=3, layout=Layout(width='260px'), options={'': None, 'Alt. division': 'signed_period_alt', 'Year': 'signed_year', 'Default division': 'signed_period'}, value='signed_period'))), VBox(children=(IntSlider(value=5, continuous_update=False, description='Top #:', max=25, min=1), FloatSlider(value=0.001, continuous_update=False, description='Threshold:', max=0.5, readout_format='.3f', step=0.01))), VBox(children=(IntProgress(value=0, max=5), Dropdown(description='Output:', index=3, layout=Layout(width='260px'), options={'': None, 'Alt. division': 'signed_period_alt', 'Year': 'signed_year', 'Default division': 'signed_period'}, value='signed_period'))))), Output()))